In [1]:
!pip install vllm
!pip install pygoogletranslation
!pip install googletrans==4.0.0-rc1

In [2]:
from vllm import LLM, SamplingParams

llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.1",trust_remote_code=True,dtype="auto")

INFO 12-16 20:30:41 llm_engine.py:73] Initializing an LLM engine with config: model='mistralai/Mistral-7B-Instruct-v0.1', tokenizer='mistralai/Mistral-7B-Instruct-v0.1', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)
INFO 12-16 20:30:52 llm_engine.py:222] # GPU blocks: 8921, # CPU blocks: 2048


**Structure Output with One shot Learning**

**Chat Template for Mistral-7B-Instruct**

`<s>[INST] Instruction [/INST] Model answer</s>[INST] Follow-up instruction [/INST]`

In [3]:
from pygoogletranslation import Translator
translator = Translator()

In [4]:
#prompts = "နေ့တစ်နေ့ကို အကျိုးရှိရှိ ဘယ်လိုစတင်မလဲ ?"
prompts = "မြန်မာနိုင်ငံမှာ စိတ်ဝင်စားစရာအကောင်းဆုံးနေရာတွေက ဘာတွေလဲ ?"
#prompts =  "ပရိုဂရမ်းမင်းကို ဘယ်လိုစလေ့လာရမလဲ ?"
#prompts = "ပုဂံအကြောင်း ပြောပြပါ?"

prompt = translator.translate(f'{prompts}', dest='en')
my_prompt = prompt.text
my_prompt = my_prompt
print(my_prompt)

What are the most interesting places in Myanmar?


In [5]:
query = my_prompt
Instruction = """
You are a helpful AI assistant. Your task is to generate a List based on the given information. So for instance the following:
Think step by step.

Do you have a morning ritual?
would be converted to:
"""

sample_answer = """
1. Wake up early (6:30 am) and get out of bed.
2. Drink a glass of water to hydrate myself.
3. Exercise for 30 minutes (usually yoga or running).
4. Take a shower to start the day fresh.
"""

user_question = f"""
{query} :
"""

prompt_template = f"<s>[INST] {Instruction} [/INST] {sample_answer} </s>[INST] {user_question} [/INST]"

In [6]:
sampling_params = SamplingParams(temperature=0.5, top_p=0.95,max_tokens=200)
outputs = llm.generate(prompt_template, sampling_params)

result = " "
for output in outputs:
    generated_text = output.outputs[0].text
    translator = Translator()
    result = translator.translate(f'{generated_text}', dest='my')



Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  3.00s/it]


In [7]:
print(f"Prompt: {prompts!r}")
print(" ")
print(f"Generated text: {result.text!r}".replace('. ', '.\n'))


Prompt: 'မြန်မာနိုင်ငံမှာ စိတ်ဝင်စားစရာအကောင်းဆုံးနေရာတွေက ဘာတွေလဲ ?'
 
Generated text: '1.
ရန်ကုန် (ယခင်ဟုလူသိများသော) ရန်ကုန် - ရွှေတိဂုံစေတီတော်နှင့်ဆူးလေဘုရားဘုရားကဲ့သို့ပင်မြန်မာနိုင်ငံ၏အကြီးဆုံးမြို့နှင့်အိမ်ရာတွင်အကြီးဆုံးမြို့။\n2.
မန္တလေးမြို့ - မန္တလေး - မြန်မာနိုင်ငံအလယ်ပိုင်းရှိမြို့တစ်မြို့သည်၎င်း၏ကြွယ်ဝသောသမိုင်း, လှပသောဘုရားကျောင်းများနှင့်စည်ကားနေသောစျေးကွက်များကြောင့်လူသိများသောမြို့တစ်မြို့ဖြစ်သည်။\n3.
ပုဂံ - မြန်မာမြောက်ပိုင်းရှိရှေးဟောင်းမြို့တစ်မြို့သည်ထောင်နှင့်ချီသောဗုဒ္ဓဘာသာ 0 င်များ, ဘုရား,\n4 ။\n5.
ဘားအံ - မြန်မာနိုင်ငံတောင်ပိုင်းရှိမြို့တစ်မြို့သည်ရင်သပ်ရှုမောဖွယ်ထုံးကျောက်များ, ဂူများနှင့်သဘာဝအလှအပများကြောင့်လူသိများသောမြို့တစ်မြို့ဖြစ်သည်။\n6 ။'
